In [1]:
import covasim as cv
import matplotlib as mpl
import matplotlib.pyplot as plt
import pylab as pl
import sciris as sc
import numpy as np
import scipy
from scipy.stats import norm
from scipy.stats import bernoulli

Covasim 3.1.4 (2022-10-22) — © 2020-2022 by IDM


In [19]:
class dem_random_masking(cv.Intervention):
  def __init__(self,mask_eff=None,maskprob_ub=None,maskprob_lb=None,mean=None,std=None,*args,**kwargs):
    super().__init__(**kwargs)
    self.mask_eff         = mask_eff # mask effectiveness
    self.maskprob_ub      = maskprob_ub # probability of masking upper bound
    self.maskprob_lb      = maskprob_lb # probability of masking lower bound
    self.mean             = mean # mean of normal distribution used for beta1 in logit fucntion
    self.std              = std # standard deviation of normal distribution used for beta1 in logit function
    self.not_mask         = [] # track number of agents not masking
    self.num_mask         = [] # track number of agents wearing mask
    self.t                = [] # track each time step
    self.avg_p_child      = [] # track average probability of child masking
    self.avg_p_adolescent = [] # track average probability of adolescent masking
    self.avg_p_adult      = [] # track average probability of adult masking
    self.avg_p_senior     = [] # track average probability of senior masking
    self.avg_p_supsenior  = [] # track average probability of supsenior masking
    self.avg_p_male       = [] # track average probabiltiy of male masking
    self.avg_p_female     = [] # track average probability of female masking
    self.avg_p            = [] # track average probability of masking
    return

  def initialize(self,sim):
    super().initialize()
    ppl             = sim.people # shorten sim.people
    self.pop        = len(ppl) # record population size
    self.child      = np.logical_and(ppl.age > 2, ppl.age <= 9) # set children as agents aged 2 to 9
    self.adolescent = np.logical_and(ppl.age > 9, ppl.age <= 19) # set adolescent as agents aged 10 to 19
    self.adult      = np.logical_and(ppl.age > 19, ppl.age <= 69) # set adult as agents aged 20 to 69
    self.senior     = np.logical_and(ppl.age > 69, ppl.age <= 79) # set senior as agents aged 70 to 79
    self.supsenior  = ppl.age > 79 # set supersenior as agents aged 79+
    self.male       = ppl.sex == 1 # set male agents
    self.female     = ppl.sex == 0 # set female agents
    return

  def apply(self,sim):
    ppl                = sim.people
    ppl.rel_sus        = ppl.rel_sus # reset all agents relative susceptibility 
    self.num_dead      = ppl.dead.sum() # record number of agents dead at timestep t
    self.num_diagnosed = (ppl.diagnosed & ppl.infectious).sum() # record number of agents diagnosed at timestep t
    self.x             = self.num_dead + self.num_diagnosed # set x equal to number of agents diagnosed and dead
    self.norm          = norm.rvs(loc=self.mean,scale=self.std,size=self.pop) # set normal distrubution array of values for beta1 in logit function
    self.norm_f        = norm.rvs(loc=1.14,scale=0.15,size=self.pop) # set normal distribution array of values for masking probability multiplier for female agents
    
    
# Masking Children
    self.p_child       = np.exp(0.001+((self.norm*0.34)*(self.x/self.pop))-0.001*sim.t) 
    self.p_child       = (self.p_child/(1+self.p_child))-0.5 # create logit function
    self.p_child       = np.clip(self.p_child,self.maskprob_lb,self.maskprob_ub) # include bounds on logit function
    self.mc_mean       = np.mean(self.p_child) # record mean of probability of masking for male children
    bernoulli_dist     = scipy.stats.bernoulli(p=self.p_child) 
    self.child_masking = bernoulli_dist.rvs(size=len(self.p_child)) # perform bernoulli using probability of masking for male children
    ppl.rel_sus        = np.where(self.child_masking & self.child & self.male,0.34*self.mask_eff,ppl.rel_sus) # change relative susceptibility values for male children that are masking

    self.p_child_f       = np.exp(0.001+((self.norm*0.34)*(self.x/self.pop))-0.001*sim.t)
    self.p_child_f       = self.norm_f*(self.p_child_f/(1+self.p_child_f))-0.5
    self.p_child_f       = np.clip(self.p_child_f,self.maskprob_lb,self.maskprob_ub)
    self.fc_mean         = np.mean(self.p_child_f)
    bernoulli_dist       = scipy.stats.bernoulli(p=self.p_child_f)
    self.child_masking_f = bernoulli_dist.rvs(size=len(self.p_child_f))
    ppl.rel_sus          = np.where(self.child_masking_f & self.child & self.female,0.34*self.mask_eff,ppl.rel_sus) # change relative susceptibility values for female children that are masking
   
    self.p_avg_c         = (self.mc_mean+self.fc_mean)/2 # record average probability of a child masking 
    self.avg_p_child.append(self.p_avg_c)
    

# Masking Adolescents
    self.p_adolescent        = np.exp(0.001+((self.norm*0.67)*(self.x/self.pop))-0.001*sim.t)
    self.p_adolescent        = (self.p_adolescent/(1+self.p_adolescent))-0.5
    self.p_adolescent        = np.clip(self.p_adolescent,self.maskprob_lb,self.maskprob_ub)
    self.mad_mean            = np.mean(self.p_adolescent)
    bernoulli_dist           = scipy.stats.bernoulli(p=self.p_adolescent)
    self.adolescent_masking  = bernoulli_dist.rvs(size=len(self.p_adolescent))
    ppl.rel_sus              = np.where(self.adolescent_masking & self.adolescent & self.male,0.67*self.mask_eff,ppl.rel_sus)

    self.p_adolescent_f       = np.exp(0.001+((self.norm*0.67)*(self.x/self.pop))-0.001*sim.t)
    self.p_adolescent_f       = self.norm_f*(self.p_adolescent_f/(1+self.p_adolescent_f))-0.5
    self.p_adolescent_f       = np.clip(self.p_adolescent_f,self.maskprob_lb,self.maskprob_ub)
    self.fad_mean             = np.mean(self.p_adolescent_f)
    bernoulli_dist           = scipy.stats.bernoulli(p=self.p_adolescent_f)
    self.adolescent_masking_f  = bernoulli_dist.rvs(size=len(self.p_adolescent_f))
    ppl.rel_sus               = np.where(self.adolescent_masking_f & self.adolescent & self.female,0.67*self.mask_eff,ppl.rel_sus)
    
    self.p_avg_ad             = (self.mad_mean+self.fad_mean)/2
    self.avg_p_adolescent.append(self.p_avg_ad)

# Masking Adults
    self.p_adult        = np.exp(0.001+((self.norm)*(self.x/self.pop))-0.001*sim.t)
    self.p_adult        = (self.p_adult/(1+self.p_adult))-0.5
    self.p_adult        = np.clip(self.p_adult,self.maskprob_lb,self.maskprob_ub)
    self.ma_mean        = np.mean(self.p_adult)
    bernoulli_dist      = scipy.stats.bernoulli(p=self.p_adult)
    self.adult_masking  = bernoulli_dist.rvs(size=len(self.p_adult))
    ppl.rel_sus         = np.where(self.adult_masking & self.adult & self.male,1.00*self.mask_eff,ppl.rel_sus)
  

    self.p_adult_f       = np.exp(0.001+((self.norm)*(self.x/self.pop))-0.001*sim.t)
    self.p_adult_f       = self.norm_f*(self.p_adult_f/(1+self.p_adult_f))-0.5
    self.p_adult_f       = np.clip(self.p_adult_f,self.maskprob_lb,self.maskprob_ub)
    self.fa_mean         = np.mean(self.p_adult_f)
    bernoulli_dist      = scipy.stats.bernoulli(p=self.p_adult_f)
    self.adult_masking_f  = bernoulli_dist.rvs(size=len(self.p_adult_f))
    ppl.rel_sus          = np.where(self.adult_masking & self.adult & self.female,1.00*self.mask_eff,ppl.rel_sus)
    
    
    self.p_avg_a         = (self.ma_mean+self.fa_mean)/2
    self.avg_p_adult.append(self.p_avg_a)

# Masking Seniors
    self.p_senior       = np.exp(0.001+((self.norm*1.24)*(self.x/self.pop))-0.001*sim.t)
    self.p_senior       = (self.p_senior/(1+self.p_senior))-0.5
    self.p_senior       = np.clip(self.p_senior,self.maskprob_lb,self.maskprob_ub)
    self.ms_mean        = np.mean(self.p_senior)
    bernoulli_dist      = scipy.stats.bernoulli(p=self.p_senior)
    self.senior_masking  = bernoulli_dist.rvs(size=len(self.p_senior))
    ppl.rel_sus         = np.where(self.senior_masking & self.senior,1.24*self.mask_eff,ppl.rel_sus)
  

    self.p_senior_f       = np.exp(0.001+((self.norm*1.24)*(self.x/self.pop))-0.001*sim.t)
    self.p_senior_f       = self.norm_f*(self.p_senior_f/(1+self.p_senior_f))-0.5
    self.p_senior_f       = np.clip(self.p_senior_f,self.maskprob_lb,self.maskprob_ub)
    self.fs_mean          = np.mean(self.p_senior_f)
    bernoulli_dist        = scipy.stats.bernoulli(p=self.p_senior_f)
    self.senior_masking_f = bernoulli_dist.rvs(size=len(self.p_senior_f))
    ppl.rel_sus           = np.where(self.senior_masking_f & self.senior,1.24*self.mask_eff,ppl.rel_sus)

 
    self.p_avg_s          = (self.ms_mean+self.fs_mean)/2
    self.avg_p_senior.append(self.p_avg_s)


# Masking Superseniors    
    self.p_supsenior       = np.exp(0.001+((self.norm*1.47)*(self.x/self.pop))-0.001*sim.t)
    self.p_supsenior       = (self.p_supsenior/(1+self.p_supsenior))-0.5
    self.p_supsenior       = np.clip(self.p_supsenior,self.maskprob_lb,self.maskprob_ub)
    self.msup_mean         = np.mean(self.p_supsenior)
    bernoulli_dist         = scipy.stats.bernoulli(p=self.p_supsenior)
    self.supsenior_masking = bernoulli_dist.rvs(size=len(self.p_supsenior))   
    ppl.rel_sus            = np.where(self.supsenior_masking & self.supsenior & self.male,1.47*self.mask_eff,ppl.rel_sus)
   

    self.p_supsenior_f       = np.exp(0.001+((self.norm*1.47)*(self.x/self.pop))-0.001*sim.t)
    self.p_supsenior_f       = self.norm_f*(self.p_supsenior_f/(1+self.p_supsenior_f))-0.5
    self.p_supsenior_f       = np.clip(self.p_supsenior_f,self.maskprob_lb,self.maskprob_ub)
    self.fsup_mean           = np.mean(self.p_supsenior_f)
    bernoulli_dist           = scipy.stats.bernoulli(p=self.p_supsenior_f)
    self.supsenior_masking_f = bernoulli_dist.rvs(size=len(self.p_supsenior_f))
    ppl.rel_sus              = np.where(self.supsenior_masking_f & self.supsenior & self.female,1.47*self.mask_eff,ppl.rel_sus)
    
    
    self.p_avg_sup           = (self.msup_mean+self.fsup_mean)/2
    self.avg_p_supsenior.append(self.p_avg_sup)

    self.total_masks = len(set(np.where(self.child_masking | self.child_masking_f)[0]) |       # count total number of agents masking at timestep t
                       set(np.where(self.adolescent_masking | self.adolescent_masking_f)[0]) |
                       set(np.where(self.adult_masking | self.adult_masking_f)[0]) |
                       set(np.where(self.senior_masking | self.senior_masking_f)[0]) |
                       set(np.where(self.supsenior_masking | self.supsenior_masking_f)[0]))
    
    self.not_masked = self.pop-self.total_masks # count number of agents not masking at timestep t
    self.p_male     = (self.mc_mean+self.mad_mean+self.ma_mean+self.ms_mean+self.msup_mean)/5 # get average probability of male agent masking
    self.p_female   = (self.fc_mean+self.fad_mean+self.fa_mean+self.fs_mean+self.fsup_mean)/5 # get average probability of female agent masking
    self.p          = (self.p_male+self.p_female)/2 # get average of all agents masking 
    self.num_mask.append(self.total_masks)
    self.not_mask.append(self.not_masked)
    self.avg_p.append(self.p)
    self.avg_p_male.append(self.p_male)
    self.avg_p_female.append(self.p_female)
    self.t.append(sim.t)
    return

def plot(self):
    plt.figure() # plot probability of masking by age
    plt.plot(self.t,self.avg_p_child,label='Children')
    plt.plot(self.t,self.avg_p_adolescent,label='Adolescents')
    plt.plot(self.t,self.avg_p_adult,label='Adults')
    plt.plot(self.t,self.avg_p_senior,label='Seniors')
    plt.plot(self.t,self.avg_p_supsenior,label='Super Seniors')
    plt.legend()
    plt.xlabel('Day')
    plt.ylabel('Avg Probability of Masking')
    plt.title('Avg Probability of Masking by Age')
    plt.show()

    plt.figure() # plot probability of masking by sex
    plt.plot(self.t,self.avg_p_male,label='Male')
    plt.plot(self.t,self.avg_p_female,label='Female')
    plt.legend()
    plt.xlabel('Day')
    plt.ylabel('Avg Probability of Masking')
    plt.title('Avg Probability of Masking by Sex')
    plt.show()

    plt.figure() # plot avg probability of masking
    plt.plot(self.t,self.avg_p)
    plt.xlabel('Day')
    plt.ylabel('Avg Probability of Masking')
    plt.title('Avg Probability of Masking')
    plt.show()

    pl.figure() # plot number of masked and not masked agents
    pl.plot(self.t,self.num_mask,label='Masked')
    pl.plot(self.t,self.not_mask,label='Not Masked')
    pl.legend()
    pl.xlabel('Day')
    pl.ylabel('# of Agents')
    pl.title('# of Agents Masking Over Time')
    pl.show()
    return

In [16]:
tp = cv.test_prob(symp_prob=0.1,asymp_prob=0.001,symp_quar_prob=0.3,asymp_quar_prob=0.3,quar_policy='daily')
dm = dem_random_masking(mask_eff=0.6,maskprob_ub=0.75,maskprob_lb=0.00,mean=100,std=50)

In [23]:

sim = cv.Sim(pop_size=200e3,n_days=180,interventions=[tp,dm],beta=0.023,label='masking')


sim.run().plot()

Initializing sim with 200000 people for 180 days
  Running "masking": 2020-03-01 ( 0/180) (1.06 s)  ———————————————————— 1%
  Running "masking": 2020-03-11 (10/180) (3.01 s)  •——————————————————— 6%
  Running "masking": 2020-03-21 (20/180) (5.40 s)  ••—————————————————— 12%
  Running "masking": 2020-03-31 (30/180) (7.22 s)  •••————————————————— 17%
  Running "masking": 2020-04-10 (40/180) (9.07 s)  ••••———————————————— 23%
  Running "masking": 2020-04-20 (50/180) (11.19 s)  •••••——————————————— 28%
  Running "masking": 2020-04-30 (60/180) (13.75 s)  ••••••—————————————— 34%
  Running "masking": 2020-05-10 (70/180) (16.73 s)  •••••••————————————— 39%
  Running "masking": 2020-05-20 (80/180) (19.98 s)  ••••••••———————————— 45%
  Running "masking": 2020-05-30 (90/180) (23.27 s)  ••••••••••—————————— 50%
  Running "masking": 2020-06-09 (100/180) (26.50 s)  •••••••••••————————— 56%
